In [52]:
import numpy as np
import re

In [11]:
def attach_names(dvals, names):
    # Check that the lengths of names and data are the same and raise an assertion error if not
    assert len(names) == len(dvals), "Length of names does not match data"
    patients = []
    for i in range(0, len(names)):
        patients.append({'name': names[i],
                         'data': dvals[i]})
    return patients

In [12]:
data = np.array([[1., 2., 3.],
                 [4., 5., 6.]])

output = attach_names(data, ['Alice', 'Bob'])
print(output)

[{'name': 'Alice', 'data': array([1., 2., 3.])}, {'name': 'Bob', 'data': array([4., 5., 6.])}]


In [ ]:
fail = attach_names(data, ['Alice', 'Bob', 'Extra'])
print(fail)

In [16]:
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author

    def __str__(self):
        return (self.title + ' by ' + self.author)


In [17]:
book = Book('A Book', 'Me')
print(book)

A Book by Me


In [34]:
class Observation:
    def __init__(self, day, value):
        self.day = day
        self.value = value

    def __str__(self):
        return str(self.value)

class Person:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

class Patient(Person):
    """A patient in an inflammation study."""
    def __init__(self, name):
        super().__init__(name)
        self.observations = []
        # status contains whether the patient is enrolled / has left study / is dead
        self.status = []

    def add_observation(self, value, day=None):
        if day is None:
            try:
                day = self.observations[-1].day + 1

            except IndexError:
                day = 0

        new_observation = Observation(day, value)

        self.observations.append(new_observation)
        return new_observation

    def add_status(self, status, day=None):
        if day is None:
            try:
                day = self.status[-1].day + 1

            except IndexError:
                day = 0

        new_status = status
        self.status.append(new_status)
        return new_status
    
class Doctor(Person):
    """A doctor who has patients."""
    def __init__(self, name):
        super().__init__(name)
        self.patients = []
    
    def add_patient(self, patient):
        # Check that the patient is not already in the doctor's list
        for p in self.patients:
            if patient.name == p.name:
                return
        self.patients.append(patient)


In [46]:
# Some tests to see if the code above works
def test_create_patient():
    name = 'Alice'
    p = Patient(name = name)
    # Check that the patient is a person
    assert isinstance(p, Person)
    # Check person's name is assigned correctly
    assert p.name == name

def test_create_doctor():
    name = 'Test Doctor'
    d = Doctor(name = name)
    assert isinstance(d, Person)
    assert d.name == name

def test_patients_added():
    d = Doctor('Test Doctor')
    p = Patient('Alice')
    d.add_patient(p)
    assert d.patients is not None
    assert len(d.patients) == 1

def test_no_duplicate_patients():
    d = Doctor('Test Doctor')
    p = Patient('Alice')
    d.add_patient(p)
    d.add_patient(p)
    assert len(d.patients) == 1

In [65]:
class Person:
    """A person."""
    def __init__(self, name):
        self.name = name
        self.id = None

    def __str__(self):
        return self.name

    def set_id(self, id):
      raise NotImplementedError('set_id not implemented')

    def get_id(self):
        return self.id

class Patient(Person):
    """A patient in an inflammation study."""
    def __init__(self, name):
        super().__init__(name)

    def __str__(self):
        return 'Patient: ' + super().__str__()

    def set_id(self, id):
        self.id = 'P' + str(id).zfill(4)


class Doctor(Person):
    """A doctor in an inflammation study."""
    def __init__(self, name):
        super().__init__(name)

    def __str__(self):
        return 'Doctor: ' + super().__str__()

    def set_id(self, id):
        self.id = 'D' + str(id).zfill(4)

class Administrator:
    """An administrator in an inflammation study."""
    def __init__(self, name):
        self.name = name
        self.id = None

    def __str__(self):
        return 'Administrator: ' + self.name

    def set_id(self, id):
        self.id = 'A' + str(id).zfill(4)

    def get_id(self):
        return self.id

In [66]:
class Trial:
    """A trial that has people (doctors, patients, administrators)."""
    def __init__(self):
        self.people = []

    def add_person(self, new_person):
        # Add a person to the trial
        if isinstance(new_person, Person):
            self.people.append(new_person)
        elif hasattr(new_person, 'get_id'):
            new_person_id = new_person.get_id()
            if re.match("^A[0-9]{4}$", new_person_id):
                new_person = Administrator(new_person_id)
            elif re.match("^D[0-9]{4}$", new_person_id):
                new_person = Doctor(new_person_id)
            elif re.match("^P[0-9]{4}$", new_person_id):
                new_person = Patient(new_person_id)
            else: raise TypeError('new person id does not begin with A, D, or P and/or is not followed by 4 numbers')
            self.people.append(new_person)
        else: raise TypeError('new person must be a Person object or have a get_id method')
    
    def print_people(self):
        # Print the names of all people in the trial (since they are all now people they have names)
        for p in self.people:
            print(p)

    def set_ids(self):
        # Assign unique ids to everyone in the trial
        for i, p in enumerate(self.people):
            if hasattr(p, 'set_id'):
                p.set_id(i + 1)
            else: raise TypeError('person does not have method to set id')

    

In [72]:
newtrial = Trial()
newtrial.add_person(Doctor('Doctor Bones'))
sheila = Administrator('Sheila')
sheila.set_id(235)
newtrial.add_person(sheila)
newtrial.add_person(Patient('Sickman'))
# newtrial.add_person('Failure')

newtrial.print_people()

# This reassigns ids
newtrial.set_ids()
for p in newtrial.people:
    print (p.id)

Doctor: Doctor Bones
Administrator: A0235
Patient: Sickman
D0001
A0002
P0003
